## Gaussian Filtering

In [1]:
from PIL import Image
import numpy as np
import math

### 1. boxfilter

In [2]:
def boxfilter(n):
    # n이 odd가 아니면 assert error
    assert n%2==1, "Dimension must be odd"
    return np.ones((n, n)) / (n * n)


In [3]:
boxfilter(3)

array([[0.11111111, 0.11111111, 0.11111111],
       [0.11111111, 0.11111111, 0.11111111],
       [0.11111111, 0.11111111, 0.11111111]])

In [4]:
boxfilter(4)

AssertionError: Dimension must be odd

In [5]:
boxfilter(5)

array([[0.04, 0.04, 0.04, 0.04, 0.04],
       [0.04, 0.04, 0.04, 0.04, 0.04],
       [0.04, 0.04, 0.04, 0.04, 0.04],
       [0.04, 0.04, 0.04, 0.04, 0.04],
       [0.04, 0.04, 0.04, 0.04, 0.04]])

### 2.gauss1d(sigma)

In [6]:
def gauss1d(sigma):
    length = math.ceil(sigma * 6)
    if length % 2 == 0:
        length += 1
    xs = np.arange(-(length // 2), length // 2 + 1, 1)
    gaussian = np.exp(-xs**2 / (2 * sigma ** 2))
    return gaussian / np.sum(gaussian)

In [7]:
gauss1d(0.3)

array([0.00383626, 0.99232748, 0.00383626])

In [8]:
gauss1d(0.5)

array([0.10650698, 0.78698604, 0.10650698])

In [9]:
gauss1d(1)

array([0.00443305, 0.05400558, 0.24203623, 0.39905028, 0.24203623,
       0.05400558, 0.00443305])

In [10]:
gauss1d(2)

array([0.0022182 , 0.00877313, 0.02702316, 0.06482519, 0.12110939,
       0.17621312, 0.19967563, 0.17621312, 0.12110939, 0.06482519,
       0.02702316, 0.00877313, 0.0022182 ])

### 3. gauss2d(sigma)

In [11]:
def gauss2d(sigma):
    gaussian1d = gauss1d(sigma)
    return np.outer(gaussian1d, gaussian1d)

In [12]:
gauss2d(0.5)

array([[0.01134374, 0.08381951, 0.01134374],
       [0.08381951, 0.61934703, 0.08381951],
       [0.01134374, 0.08381951, 0.01134374]])

In [13]:
gauss2d(1)

array([[1.96519161e-05, 2.39409349e-04, 1.07295826e-03, 1.76900911e-03,
        1.07295826e-03, 2.39409349e-04, 1.96519161e-05],
       [2.39409349e-04, 2.91660295e-03, 1.30713076e-02, 2.15509428e-02,
        1.30713076e-02, 2.91660295e-03, 2.39409349e-04],
       [1.07295826e-03, 1.30713076e-02, 5.85815363e-02, 9.65846250e-02,
        5.85815363e-02, 1.30713076e-02, 1.07295826e-03],
       [1.76900911e-03, 2.15509428e-02, 9.65846250e-02, 1.59241126e-01,
        9.65846250e-02, 2.15509428e-02, 1.76900911e-03],
       [1.07295826e-03, 1.30713076e-02, 5.85815363e-02, 9.65846250e-02,
        5.85815363e-02, 1.30713076e-02, 1.07295826e-03],
       [2.39409349e-04, 2.91660295e-03, 1.30713076e-02, 2.15509428e-02,
        1.30713076e-02, 2.91660295e-03, 2.39409349e-04],
       [1.96519161e-05, 2.39409349e-04, 1.07295826e-03, 1.76900911e-03,
        1.07295826e-03, 2.39409349e-04, 1.96519161e-05]])

### 4. convovle2d(array, filter)

In [14]:
def convolve2d(array, filter):
    height, width = array.shape
    filterSize = filter.shape[0]

    paddingSize = filterSize // 2

    modifiedArray = np.zeros((height + 2 * paddingSize, width + 2 * paddingSize))
    modifiedArray[paddingSize:-paddingSize, paddingSize: -paddingSize] = array

    result = np.zeros((height, width))

    np.flip(filter, axis=0)
    np.flip(filter, axis=1)

    for i in range(height):
        for j in range(width):
            window = modifiedArray[i:i+filterSize, j:j+filterSize]

            result[i,j] = np.sum(window * filter)
    
    return result
    

In [15]:
def gaussconvolve2d(array, sigma):
    filter = gauss2d(sigma)
    return convolve2d(array, filter)

In [16]:
im = Image.open('./images/2b_dog.bmp')
im.show()
im = im.convert('L')
im = np.asarray(im)

im = gaussconvolve2d(im, 3)
im = np.clip(im, 0, 255).astype(np.uint8)
im = Image.fromarray(im)
im.save("./result_images/dog_convolution.png", "PNG")
im.show()

## Hypbrid Images

### 1. Gaussian filtered low frequency image

In [17]:
blur = Image.open('./images/2b_dog.bmp')
blur = np.asarray(blur)
sigma = 10

r = gaussconvolve2d(blur[:,:,0], sigma)
g = gaussconvolve2d(blur[:,:,1], sigma)
b = gaussconvolve2d(blur[:,:,2], sigma)

blur = np.dstack([r, g, b])
blur = np.clip(blur, 0, 255).astype(np.uint8)
blurImage = Image.fromarray(blur)
blurImage.save("./result_images/blur_image.png", "PNG")
blurImage.show()

In [18]:
sharpen = Image.open('./images/2a_cat.bmp')
sharpen = np.asarray(sharpen)
sigma = 10

r = gaussconvolve2d(sharpen[:,:,0], sigma)
g = gaussconvolve2d(sharpen[:,:,1], sigma)
b = gaussconvolve2d(sharpen[:,:,2], sigma)

blurredSharpen = np.dstack([r, g, b])
blurredSharpen = np.clip(blurredSharpen, 0, 255).astype(np.int16)

highFreqSharpen = sharpen - blurredSharpen

highFreqSharpen = np.clip(highFreqSharpen, -128, 127).astype(np.int16)

visulalizeHighFreqSharpen = highFreqSharpen + np.ones_like(highFreqSharpen) * 128
visulalizeHighFreqSharpen = np.clip(visulalizeHighFreqSharpen, 0, 255).astype(np.uint8)
visulalizeHighFreqSharpen = Image.fromarray(visulalizeHighFreqSharpen)

visulalizeHighFreqSharpen.save("./result_images/high_freq_image.png", "PNG")
visulalizeHighFreqSharpen.show()

In [19]:
hybridImage = blur + highFreqSharpen
hybridImage = np.clip(hybridImage, 0, 255).astype(np.uint8)
hybridImage = Image.fromarray(hybridImage)
hybridImage.save("./result_images/hybrid_image.png", "PNG")
hybridImage.show()